In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d 'charlesdometian/cassava-plant-diseases'

Dataset URL: https://www.kaggle.com/datasets/charlesdometian/cassava-plant-diseases
License(s): unknown
100% 2.38G/2.39G [00:36<00:00, 151MB/s]
100% 2.39G/2.39G [00:36<00:00, 69.5MB/s]


In [ ]:
# Import library yang dibutuhkan
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from hyperopt import fmin, tpe, hp
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom
from keras.layers import Flatten
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
#Melakukan ekstrak pada dataset
local_zip = '/content/cassava-plant-diseases.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
base_dir = 'cassava_dataset'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [ ]:
# Membuat direktori base jika belum ada
if not os.path.exists(base_dir):
    os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

# Membuat direktori train dan validation
os.mkdir(train_dir)
os.mkdir(validation_dir)

In [ ]:
Cassava___bacterial_blight_dir = os.path.join(base_dir, 'Cassava___bacterial_blight')
Cassava___brown_streak_disease_dir = os.path.join(base_dir,'Cassava___brown_streak_disease')
Cassava___green_mottle_dir = os.path.join(base_dir,'Cassava___green_mottle')
Cassava___healthy_dir = os.path.join(base_dir,'Cassava___healthy')
Cassava___mosaic_disease_dir = os.path.join(base_dir,'Cassava___mosaic_disease')

In [ ]:
#Memecah direktori cassava_bacterial_blight menjadi data train dan data validation
train_Cassava___bacterial_blight_dir, val_Cassava___bacterial_blight_dir = train_test_split(os.listdir(Cassava___bacterial_blight_dir), test_size = 0.1)

#Memecah direktori cassava_brown_streak_disease menjadi data train dan data validation
train_Cassava___brown_streak_disease_dir, val_Cassava___brown_streak_disease_dir = train_test_split(os.listdir(Cassava___brown_streak_disease_dir), test_size = 0.1)

#Memecah direktori cassava_green_mottle menjadi data train dan data validation
train_Cassava___green_mottle_dir, val_Cassava___green_mottle_dir = train_test_split(os.listdir(Cassava___green_mottle_dir), test_size = 0.1)

#Memecah direktori cassava_healthy menjadi data train dan data validation
train_Cassava___healthy_dir, val_Cassava___healthy_dir = train_test_split(os.listdir(Cassava___healthy_dir), test_size = 0.1)

#Memecah direktori cassava_mosaic_disease menjadi data train dan data validation
train_Cassava___mosaic_disease_dir, val_Cassava___mosaic_disease_dir = train_test_split(os.listdir(Cassava___mosaic_disease_dir), test_size = 0.1)

In [ ]:
#Mendefinisikan tiap folder untuk setiap train
train_Cassava___bacterial_blight = os.path.join(train_dir, 'Cassava___bacterial_blight')
train_Cassava___brown_streak_disease = os.path.join(train_dir, 'Cassava___brown_streak_disease')
train_Cassava___green_mottle = os.path.join(train_dir, 'Cassava___green_mottle')
train_Cassava___healthy = os.path.join(train_dir, 'Cassava___healthy')
train_Cassava___mosaic_disease = os.path.join(train_dir, 'Cassava___mosaic_disease')

#Mendefinisikan tiap folder untuk setiap validation
val_Cassava___bacterial_blight = os.path.join(validation_dir, 'Cassava___bacterial_blight')
val_Cassava___brown_streak_disease = os.path.join(validation_dir, 'Cassava___brown_streak_disease')
val_Cassava___green_mottle = os.path.join(validation_dir, 'Cassava___green_mottle')
val_Cassava___healthy = os.path.join(validation_dir, 'Cassava___healthy')
val_Cassava___mosaic_disease = os.path.join(validation_dir, 'Cassava___mosaic_disease')

# Membuat directory baru di dalam directory train
if not os.path.exists(train_Cassava___bacterial_blight):
  os.mkdir(train_Cassava___bacterial_blight)
if not os.path.exists(train_Cassava___brown_streak_disease):
  os.mkdir(train_Cassava___brown_streak_disease)
if not os.path.exists(train_Cassava___green_mottle):
  os.mkdir(train_Cassava___green_mottle)
if not os.path.exists(train_Cassava___healthy):
  os.mkdir(train_Cassava___healthy)
if not os.path.exists(train_Cassava___mosaic_disease):
  os.mkdir(train_Cassava___mosaic_disease)

# Membuat directory baru di dalam directory validation
if not os.path.exists(val_Cassava___bacterial_blight):
  os.mkdir(val_Cassava___bacterial_blight)
if not os.path.exists(val_Cassava___brown_streak_disease):
  os.mkdir(val_Cassava___brown_streak_disease)
if not os.path.exists(val_Cassava___green_mottle):
  os.mkdir(val_Cassava___green_mottle)
if not os.path.exists(val_Cassava___healthy):
  os.mkdir(val_Cassava___healthy)
if not os.path.exists(val_Cassava___mosaic_disease):
  os.mkdir(val_Cassava___mosaic_disease)

In [ ]:
# Mengcopy data train ke directory baru
for i in train_Cassava___bacterial_blight_dir:
  shutil.copy(os.path.join(Cassava___bacterial_blight_dir, i), os.path.join(train_Cassava___bacterial_blight, i))
for i in train_Cassava___brown_streak_disease_dir:
  shutil.copy(os.path.join(Cassava___brown_streak_disease_dir, i), os.path.join(train_Cassava___brown_streak_disease, i))
for i in train_Cassava___green_mottle_dir:
  shutil.copy(os.path.join(Cassava___green_mottle_dir, i), os.path.join(train_Cassava___green_mottle, i))
for i in train_Cassava___healthy_dir:
  shutil.copy(os.path.join(Cassava___healthy_dir, i), os.path.join(train_Cassava___healthy, i))
for i in train_Cassava___mosaic_disease_dir:
  shutil.copy(os.path.join(Cassava___mosaic_disease_dir, i), os.path.join(train_Cassava___mosaic_disease, i))

# Mengcopy data validation ke directory baru
for i in val_Cassava___bacterial_blight_dir:
  shutil.copy(os.path.join(Cassava___bacterial_blight_dir, i), os.path.join(val_Cassava___bacterial_blight, i))
for i in val_Cassava___brown_streak_disease_dir:
  shutil.copy(os.path.join(Cassava___brown_streak_disease_dir, i), os.path.join(val_Cassava___brown_streak_disease, i))
for i in val_Cassava___green_mottle_dir:
  shutil.copy(os.path.join(Cassava___green_mottle_dir, i), os.path.join(val_Cassava___green_mottle, i))
for i in val_Cassava___healthy_dir:
  shutil.copy(os.path.join(Cassava___healthy_dir, i), os.path.join(val_Cassava___healthy, i))
for i in val_Cassava___mosaic_disease_dir:
  shutil.copy(os.path.join(Cassava___mosaic_disease_dir, i), os.path.join(val_Cassava___mosaic_disease, i))

In [ ]:
# Callback untuk menghentikan training berdasarkan akurasi
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy') > 0.90):
      print("\nAkurasi melewati 90%, hentikan proses training!")
      self.model.stop_training = True

callback = MyCallback()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

#calculate steps per epoch and validation steps
steps_per_epoch = max(train_generator.samples//train_generator.batch_size,1)
validation_steps = max(validation_generator.samples//validation_generator.batch_size,1)

Found 19256 images belonging to 5 classes.
Found 2141 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3))

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(512, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=5
)

Epoch 1/5
601/601 [==============================] - 4755s 8s/step - loss: 0.9964 - accuracy: 0.6768 - val_loss: 21.5176 - val_accuracy: 0.4176
Epoch 2/5
601/601 [==============================] - 4784s 8s/step - loss: 0.7648 - accuracy: 0.7362 - val_loss: 2.9837 - val_accuracy: 0.6719
Epoch 3/5
204/601 [=========>....................] - ETA: 51:27 - loss: 0.6450 - accuracy: 0.7685

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(512, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze the base_model and fine-tune the model
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=20,
    callbacks = [callback]
)

In [ ]:
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = layers.Dense(1024, activation='relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.3)(x)
# x = layers.Dense(512, activation='relu')(x)
# predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=predictions)

# # Freeze the base_model
# for layer in base_model.layers:
#     layer.trainable = False

# model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# history = model.fit(
#     train_generator,
#     steps_per_epoch=steps_per_epoch,
#     validation_data=validation_generator,
#     validation_steps=validation_steps,
#     epochs=5
# )

In [ ]:
# train_acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']

# plt.figure(figsize=(12,8))
# plt.subplot(1,2,1)
# plt.plot(train_acc, label='Training Acc')
# plt.plot(val_acc, label='Validation Acc')
# plt.title("Training and Validation Acc")
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.figure(figsize=(12,8))
# plt.subplot(1,2,2)
# plt.plot(train_loss, label='Training Loss')
# plt.plot(val_loss, label='Validation Loss')
# plt.title("Training and Validation Loss")
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()

# plt.tight_layout()
# plt.show()

In [ ]:
# from google.colab import files
# uploaded = files.upload()
# for fn in uploaded.keys():

#   path = fn
#   img = image.load_img(path, target_size =(150,150))
#   imgplot = plt.imshow(img)
#   x = image.img_to_array(img)
#   x = np.expand_dims(x, axis=0)

#   images = np.vstack([x])
#   classes = model.predict(images, batch_size=32)

In [ ]:
# test_img = image.load_img(path, target_size=(150,150))
# img_arr = image.img_to_array(test_img)
# img_arr = np.expand_dims(img_arr, axis=0)
# #normalisasi piksel [0-1]
# img_arr /= 255.0

# prediksi = model.predict(img_arr)
# kelas = np.argmax(prediksi)
# # print(f"Predicted kelas : {kelas}")
# # print(f"confidence : {prediksi[0][kelas]}" )

In [ ]:
# list_kelas = list(train_generator.class_indices.keys())
# print(list_kelas)

In [ ]:
# def hasil_prediksi(kelas):
#   if kelas == 1:
#     print("Tanaman anda terkena penyakit hawar bakteri")
#     print("Akurasi prediksi",round((prediksi[0][kelas] * 100),2),"%")
#   elif kelas == 2:
#     print("Tanaman anda terkena penyakit garis-garis coklat")
#     print("Akurasi prediksi",round((prediksi[0][kelas] * 100),2),"%")
#   elif kelas == 3:
#     print("Tanaman anda terkena penyakit belang hijau")
#     print("Akurasi prediksi",round((prediksi[0][kelas] * 100),2),"%")
#   elif kelas == 4:
#     print("Tanaman anda sehat, terus perhatikan kesehatan tanaman anda!")
#     print("Akurasi prediksi",round((prediksi[0][kelas] * 100),2),"%")
#   else:
#     print("Tanaman anda terkena penyakit mosaik")
#     print("Akurasi prediksi",round((prediksi[0][kelas] * 100),2),"%")

#     return hasil_prediksi

In [ ]:
hasil_prediksi(kelas)

In [ ]:
export_dir = 'cassava_model'

model.save(export_dir)

In [ ]:
mode = "Speed"

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

In [ ]:
# Use the TFLiteConverter SavedModel API to initialize the converter
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimizations
converter.optimizations = [optimization]

# Invoke the converter to finally generate the TFLite model
tflite_model = converter.convert()

In [ ]:
import pathlib
tflite_model_file = pathlib.Path('/content/cassava_model.tflite')
tflite_model_file.write_bytes(tflite_model)